Off-the-shelf method using Surprise library from scikit for collaborative filtering.

The Surprise library offers matrix factorization-based prediction algorithms such as SVD, SVD++, NMF.

In [1]:
import numpy as np
from surprise import Dataset, evaluate, Reader, accuracy
from surprise.prediction_algorithms import predictions, matrix_factorization
from surprise.model_selection import cross_validate, GridSearchCV

In [2]:
# load train and test data
r = Reader(sep='\t')
train_data = Dataset.load_from_file('data/train.txt', reader=r)
train_Y = train_data.build_full_trainset()

test_data = Dataset.load_from_file('data/test.txt', reader=r)
test_Y = test_data.build_full_trainset()
test_data = test_Y.build_testset()

In [31]:
k = 20
eta = 0.1
reg = 0.1

# SVD
svd = matrix_factorization.SVD(n_factors=k, lr_all=eta, reg_all=reg, verbose=True)

# SVD++ algorithm aka extension of SVD taking into account implicit ratings
svdpp = matrix_factorization.SVDpp(n_factors=k, lr_all=eta, reg_all=reg, verbose=True)

# Non-negative Matrix factorization
nmf = matrix_factorization.NMF(n_factors=k, biased=True, lr_bu=eta, lr_bi=eta,
                               reg_pu=reg, reg_qi=reg,reg_bu=reg, reg_bi=reg, verbose=True)
# Fit and train, compute root mean square error
for model in [svd, svdpp, nmf]:
    model.fit(train_Y)
    print(model)
    accuracy.rmse(model.test(test_data)) # Then compute RMSE
    print()
    
# Results: (SVD algo had best RMSE)
# <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x0000015C1957C240>
# RMSE: 0.9527, epochs: 19

# <surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x0000015C1957C9E8>
# RMSE: 0.9640, epochs: 19

# <surprise.prediction_algorithms.matrix_factorization.NMF object at 0x0000015C1957CBE0>
# RMSE: 1.3586, epochs: 49

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
RMSE: 0.9527

 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19
RMSE: 0.9640

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
P

In [33]:
k = 20
eta = 0.1
reg = 0.1

model = matrix_factorization.SVD(n_factors=k, lr_all=eta, reg_all=reg, verbose=True)
model.fit(train_Y)
accuracy.rmse(model.test(test_data)) # epochs: 19, RMSE: 0.9510756520444057 on predictions

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
RMSE: 0.9511


0.9510756520444057

In [34]:
'''
This function performs matrix factorization.
Input:
    Y_train: training labels
    test_set: test set
Output: 
    newU: The 2D version of U
    newV: The 2D version of V
'''
def matrix_factor(model, train_Y):
    """ Uses matrix factorization-based algorithm SVD++
    and returns U (k x m matrix) and V (k x n matrix)
    where k in the number of factors of the model. """    
    # get user factors
    U = model.pu    
    print("U shape: " + str(U.shape)) # (m, k) = (943, 20)

    # transpose item factors
    V = np.transpose(model.qi)    
    print("V shape: " + str(V.shape)) # (k, n) = (20, 1682)
    
    return U, V
    
def SVD(U, V):
    """ Applies SVD to V and uses the ﬁrst two columns of A
    to project U,V into 2-D space. """    
    # get A matrix
    A, _, _ = np.linalg.svd(V) 
    # take first two columns of A
    A = A[:, [0, 1]] # (k, 2) = (5, 2)
    print("A shape: " + str(A.shape))

    # project U into 2-D space
    U_tilda = np.transpose(np.dot(np.transpose(A), np.transpose(U)))
    print("U shape: " + str(U_tilda.shape)) # (943, 2)

    # project V into 2-D space
    V_tilda = np.transpose(np.dot(np.transpose(A), V))
    print("V shape: " + str(V_tilda.shape)) # (1682, 2)

    return U_tilda, V_tilda

In [35]:
U, V = matrix_factor(model, train_Y)

U shape: (943, 20)
V shape: (20, 1668)


In [36]:
U_tilda, V_tilda = SVD(U, V)

A shape: (20, 2)
U shape: (943, 2)
V shape: (1668, 2)


In [14]:
def get_err(U, V, Y, A, B, y_mean, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    # Compute mean squared error on each data point in Y; include
    # regularization penalty in error calculations.
    # We first compute the total squared squared error
    err = 0.0
    for (i,j,Yij) in Y:
        print(i)
        print(j)
        print(Yij)
        print("**")
        err += 0.5 *(Yij - y_mean - np.dot(U[i-1], V[:,j-1]) - A[i-1] - B[:,j-1])**2
    # Add error penalty due to regularization if regularization
    # parameter is nonzero
    if reg != 0:
        U_frobenius_norm = np.linalg.norm(U, ord='fro')
        V_frobenius_norm = np.linalg.norm(V, ord='fro')
        A_frobenius_norm = np.linalg.norm(A, ord='fro')
        B_frobenius_norm = np.linalg.norm(B, ord='fro')
        err += 0.5 * reg * (U_frobenius_norm ** 2)
        err += 0.5 * reg * (V_frobenius_norm ** 2)
        err += 0.5 * reg * (A_frobenius_norm ** 2)
        err += 0.5 * reg * (B_frobenius_norm ** 2)
    # Return the mean of the regularized error
    return err / float(len(Y))

In [8]:
Y_train = np.loadtxt('data/train.txt').astype(int)
y_mean = np.mean(np.array([tup[2] for tup in Y_train]))

In [10]:
model.bu = model.bu.reshape(943,1)
model.bi = model.bi.reshape(1668,1)

In [15]:
err = get_err(U, V, Y_train, model.bu, np.transpose(model.bi), y_mean, reg=0.1)

196
242
3
**
186
302
3
**
22
377
1
**
166
346
1
**
298
474
4
**
115
265
2
**
253
465
5
**
305
451
3
**
6
86
3
**
62
257
2
**
286
1014
5
**
200
222
5
**
210
40
3
**
224
29
3
**
122
387
5
**
194
274
2
**
291
1042
4
**
234
1184
2
**
119
392
4
**
167
486
4
**
299
144
4
**
291
118
2
**
95
546
2
**
38
95
5
**
102
768
2
**
63
277
4
**
160
234
5
**
301
98
4
**
225
193
4
**
290
88
4
**
97
194
3
**
157
274
4
**
181
1081
1
**
278
603
5
**
276
796
1
**
7
32
4
**
10
16
4
**
284
304
4
**
201
979
2
**
276
564
3
**
287
327
5
**
246
201
5
**
249
241
5
**
178
332
3
**
251
100
4
**
81
432
2
**
260
322
4
**
25
181
5
**
72
679
2
**
87
384
4
**
290
143
5
**
42
423
5
**
292
515
4
**
115
20
3
**
20
288
1
**
201
219
4
**
13
526
3
**
138
26
5
**
167
232
1
**
60
427
5
**
57
304
5
**
189
512
4
**
243
15
3
**
92
1049
1
**
246
416
3
**
194
165
4
**
178
248
4
**
254
1444
3
**
293
5
3
**
127
229
5
**
225
480
5
**
276
54
3
**
291
144
5
**
222
366
4
**
267
518
5
**
42
403
3
**
11
111
4
**
95
625
4
**
8
338
4
**
87
1016

284
269
4
**
106
526
4
**
121
180
3
**
62
86
2
**
291
418
4
**
84
1033
4
**
293
380
2
**
207
58
3
**
194
187
4
**
109
97
3
**
283
845
4
**
297
275
5
**
181
334
1
**
78
255
4
**
11
425
4
**
308
59
4
**
193
1078
4
**
297
234
3
**
87
585
4
**
250
204
2
**
8
50
5
**
186
148
4
**
312
692
4
**
91
683
3
**
5
454
1
**
291
376
3
**
175
127
5
**
145
737
2
**
7
644
5
**
276
419
5
**
83
210
5
**
102
524
3
**
153
174
1
**
62
302
3
**
49
995
3
**
268
298
3
**
207
554
2
**
313
616
5
**
286
44
3
**
279
168
5
**
62
59
4
**
254
219
1
**
83
97
4
**
63
100
5
**
297
233
2
**
90
945
5
**
42
98
4
**
303
393
4
**
104
299
3
**
94
792
4
**
184
98
4
**
293
708
3
**
248
589
4
**
18
950
3
**
217
27
1
**
200
892
4
**
201
148
1
**
296
222
5
**
7
662
3
**
196
381
4
**
69
427
3
**
256
472
4
**
128
182
4
**
151
747
3
**
7
171
3
**
286
85
5
**
172
220
4
**
308
516
4
**
190
974
2
**
82
756
1
**
308
436
4
**
59
235
1
**
64
1063
3
**
145
756
2
**
220
298
4
**
21
324
4
**
285
269
4
**
207
65
3
**
198
658
3
**
220
333
3
**
2

**
69
174
5
**
195
265
4
**
121
509
5
**
279
509
3
**
49
17
2
**
7
196
5
**
280
472
2
**
221
780
3
**
175
96
3
**
180
431
4
**
311
1222
3
**
44
120
4
**
318
257
5
**
59
588
2
**
320
117
4
**
256
939
5
**
310
24
4
**
236
265
2
**
83
139
3
**
280
128
3
**
43
52
4
**
18
494
3
**
303
87
3
**
91
427
4
**
318
631
4
**
275
258
3
**
97
482
5
**
174
160
5
**
268
470
3
**
188
769
2
**
7
44
5
**
158
85
4
**
256
765
4
**
221
69
4
**
232
175
5
**
159
685
4
**
175
71
4
**
254
624
2
**
326
22
4
**
303
291
3
**
270
53
4
**
181
1001
1
**
254
418
3
**
56
235
1
**
11
190
3
**
162
181
4
**
117
829
3
**
268
52
3
**
320
177
5
**
6
294
2
**
210
380
4
**
151
969
5
**
42
684
4
**
62
365
2
**
207
121
3
**
26
455
3
**
234
705
5
**
270
466
5
**
97
484
3
**
11
660
3
**
305
923
5
**
67
1095
4
**
213
12
5
**
268
684
3
**
36
883
5
**
100
321
1
**
269
729
2
**
131
100
5
**
308
298
5
**
14
709
5
**
284
305
4
**
191
752
3
**
222
29
3
**
201
421
2
**
303
1315
3
**
52
1086
4
**
305
529
5
**
223
318
4
**
22
79
4
**
137
546

2
**
43
286
4
**
181
7
4
**
297
574
1
**
201
651
4
**
320
99
4
**
94
180
5
**
235
85
4
**
305
131
3
**
234
229
4
**
328
591
3
**
328
754
4
**
258
323
4
**
3
323
2
**
16
70
4
**
200
265
5
**
292
10
5
**
155
321
4
**
106
213
4
**
200
586
4
**
305
216
5
**
279
1113
3
**
178
984
2
**
331
133
3
**
167
1306
5
**
151
191
3
**
326
168
3
**
297
443
2
**
191
288
3
**
284
258
4
**
5
267
4
**
150
325
1
**
257
59
5
**
145
443
3
**
158
38
4
**
152
716
5
**
232
638
5
**
109
930
3
**
243
660
4
**
57
744
5
**
145
1057
1
**
235
275
5
**
181
124
1
**
145
182
5
**
249
476
3
**
44
11
3
**
194
566
4
**
109
218
4
**
49
10
3
**
269
210
1
**
87
233
4
**
314
791
4
**
292
132
4
**
7
300
4
**
291
460
5
**
292
176
5
**
290
1028
3
**
122
427
3
**
17
151
4
**
59
47
5
**
29
689
2
**
274
411
3
**
190
340
1
**
213
50
5
**
321
131
4
**
221
1314
3
**
195
100
5
**
236
187
3
**
92
619
4
**
303
576
3
**
42
210
5
**
246
423
3
**
181
823
2
**
197
231
3
**
181
369
3
**
130
172
5
**
276
1131
3
**
252
742
4
**
221
1067
3
**
292


294
4
**
168
411
1
**
72
204
4
**
144
523
5
**
303
398
1
**
128
215
3
**
320
11
4
**
267
684
4
**
60
490
4
**
189
694
4
**
116
905
2
**
249
240
4
**
110
300
3
**
201
1063
3
**
180
121
5
**
87
1072
3
**
6
209
4
**
63
301
5
**
179
895
5
**
148
98
3
**
13
312
1
**
15
278
1
**
176
305
5
**
102
66
3
**
293
251
4
**
42
204
5
**
328
523
5
**
279
67
4
**
158
42
3
**
70
151
3
**
271
661
4
**
37
222
3
**
279
1095
1
**
103
144
4
**
128
1141
4
**
336
577
1
**
275
191
4
**
95
173
5
**
87
651
4
**
21
678
2
**
145
1217
2
**
13
860
1
**
312
676
3
**
102
67
1
**
325
506
5
**
221
1073
4
**
2
297
4
**
305
733
3
**
275
969
2
**
11
215
3
**
341
876
4
**
231
126
5
**
269
474
4
**
102
809
3
**
254
240
1
**
234
486
3
**
256
932
3
**
249
58
5
**
305
947
4
**
262
15
3
**
325
187
3
**
11
428
4
**
40
258
3
**
313
740
2
**
276
1314
3
**
101
1051
2
**
236
699
4
**
207
134
4
**
125
945
5
**
120
282
4
**
293
461
2
**
160
93
5
**
298
418
4
**
326
444
4
**
246
849
1
**
278
301
2
**
166
288
3
**
328
4
3
**
70
265
4
**
2

...

**
210
230
3
**
788
302
4
**
452
52
3
**
764
496
5
**
712
787
3
**
423
344
4
**
625
380
3
**
661
161
4
**
707
952
3
**
709
781
3
**
733
221
4
**
271
238
4
**
699
597
3
**
119
286
5
**
749
595
4
**
529
258
4
**
474
385
4
**
479
629
3
**
392
197
5
**
230
205
3
**
380
109
2
**
432
845
4
**
385
383
1
**
387
68
4
**
456
366
2
**
806
254
3
**
747
967
3
**
428
286
3
**
10
276
4
**
453
369
2
**
434
928
5
**
332
983
2
**
641
427
4
**
595
298
4
**
541
417
4
**
370
134
4
**
675
344
4
**
674
763
5
**
798
563
2
**
655
195
3
**
774
834
1
**
416
182
4
**
697
129
5
**
774
186
3
**
796
728
3
**
451
991
2
**
796
1269
5
**
760
819
1
**
16
15
5
**
314
939
4
**
771
707
4
**
775
264
4
**
665
620
3
**
484
829
2
**
115
48
5
**
840
152
4
**
606
662
4
**
318
968
3
**
654
736
5
**
524
1065
1
**
152
775
4
**
545
391
2
**
661
70
4
**
536
486
4
**
796
1001
2
**
710
1101
4
**
561
56
5
**
43
411
3
**
763
879
3
**
567
177
4
**
416
1264
4
**
629
327
3
**
637
118
1
**
406
655
3
**
533
988
2
**
7
470
3
**
715
90
5
**
526

**
472
1239
5
**
405
626
1
**
774
300
2
**
270
88
5
**
484
71
2
**
648
123
4
**
606
652
3
**
747
639
5
**
263
357
5
**
778
238
3
**
626
268
4
**
303
388
2
**
450
794
5
**
87
273
3
**
326
510
5
**
720
333
4
**
712
59
2
**
443
358
1
**
684
252
4
**
531
890
1
**
758
837
4
**
782
902
2
**
655
96
3
**
825
276
1
**
450
951
4
**
606
1047
2
**
198
229
3
**
308
365
3
**
109
288
5
**
807
584
4
**
682
518
4
**
235
207
4
**
666
70
4
**
637
124
3
**
668
82
4
**
639
237
1
**
658
50
4
**
516
212
4
**
745
100
5
**
615
792
4
**
727
121
4
**
656
338
3
**
735
269
3
**
64
186
4
**
221
1011
4
**
332
73
4
**
201
144
4
**
780
423
5
**
670
657
5
**
837
476
3
**
566
403
3
**
345
14
4
**
60
131
4
**
766
434
5
**
778
780
3
**
327
333
2
**
85
526
4
**
468
192
4
**
670
479
5
**
192
7
4
**
23
235
1
**
606
1010
3
**
746
181
5
**
813
294
1
**
472
496
4
**
334
238
4
**
380
479
4
**
566
763
4
**
642
41
3
**
371
77
5
**
625
640
3
**
727
248
5
**
471
465
5
**
606
1151
3
**
90
65
4
**
334
301
2
**
621
735
4
**
374
38
4
**

3
**
102
396
2
**
835
628
3
**
789
93
4
**
655
125
2
**
804
228
4
**
207
223
3
**
653
121
4
**
90
402
5
**
567
32
5
**
276
993
3
**
385
484
4
**
435
28
3
**
788
798
2
**
506
187
5
**
468
367
4
**
643
824
3
**
23
216
4
**
694
604
4
**
493
91
3
**
334
56
4
**
637
873
1
**
688
338
5
**
669
490
5
**
182
48
3
**
830
288
1
**
851
27
4
**
680
285
5
**
805
1232
3
**
634
922
4
**
649
147
4
**
840
465
4
**
425
124
2
**
378
623
3
**
524
195
2
**
121
298
2
**
671
147
1
**
406
152
2
**
313
504
5
**
276
518
4
**
339
404
4
**
840
166
5
**
763
283
4
**
837
225
3
**
26
252
3
**
178
607
3
**
168
744
5
**
621
1047
3
**
301
4
4
**
13
197
4
**
805
147
5
**
303
319
5
**
342
216
5
**
804
1178
3
**
815
131
2
**
119
70
3
**
595
1165
1
**
194
173
5
**
425
976
1
**
821
275
5
**
291
3
3
**
838
181
5
**
43
222
4
**
693
98
4
**
450
87
5
**
758
100
5
**
96
183
4
**
405
288
5
**
837
274
4
**
392
260
1
**
699
820
2
**
419
79
4
**
472
94
5
**
87
535
4
**
77
96
3
**
794
514
5
**
60
656
4
**
442
204
3
**
418
333
5
**
623

3
**
418
750
2
**
790
230
4
**
536
404
4
**
622
396
1
**
772
332
4
**
619
17
1
**
852
323
3
**
707
602
4
**
658
488
4
**
49
1075
2
**
756
3
1
**
666
255
4
**
690
237
4
**
653
597
4
**
437
629
3
**
766
95
3
**
373
433
3
**
793
109
4
**
768
332
4
**
201
212
4
**
715
232
4
**
835
610
5
**
272
208
4
**
429
847
3
**
543
663
4
**
689
121
5
**
661
194
5
**
552
405
3
**
643
211
4
**
776
569
3
**
535
79
3
**
543
397
3
**
633
1046
4
**
786
240
1
**
851
255
3
**
412
211
4
**
607
212
3
**
833
614
2
**
381
462
4
**
836
507
4
**
851
544
4
**
554
684
4
**
722
678
3
**
280
111
4
**
823
233
4
**
340
15
5
**
391
148
3
**
666
616
3
**
499
153
4
**
363
186
3
**
405
60
1
**
455
660
4
**
650
69
2
**
773
171
5
**
582
475
5
**
747
929
3
**
5
189
5
**
846
1411
4
**
702
1127
2
**
476
56
4
**
690
233
3
**
268
201
3
**
393
184
4
**
104
250
3
**
632
1
3
**
707
647
5
**
494
86
3
**
83
820
2
**
639
651
4
**
782
532
2
**
389
588
5
**
794
557
4
**
436
1227
2
**
587
323
4
**
280
385
5
**
507
691
5
**
452
127
5
**
567
4

IndexError: index 1668 is out of bounds for axis 1 with size 1668

In [30]:
V.shape

(20, 1668)

In [19]:
unrated = [1349,1363,1458,1541,1541,1561,1576,1579,1581,1606,1633,1651,1666,1667,1678 ]

In [25]:
indices = [i for i in range(len(Y_train)) if Y_train[i][1] in unrated ]

In [26]:
indices

[]

In [29]:
Y_train.shape

(90000, 3)

In [45]:
for i in range(len(Y_train)):
    if Y_train[i][1] == 1669:
        print("hi")

hi


In [43]:
type(Y_train[0][1])

numpy.int32